## Imports

In [6]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from numpy import sin, cos
from matplotlib.gridspec import GridSpec
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from tqdm.notebook import tqdm

## General variables

In [39]:
# Constants
g = 9.8  # [m/s^2]
L1 = 5.0 # [m]
L2 = 1.0 # [m]

# Parameters
t_max = 10.0  # [s]
dt = 0.01  # [s]
m1 = 1.0  # [kg]
m2 = 1.5  # [kg]
A = m2/m1
B = L2/L1
C = g/L1  # [s^(-2)]

# Variables
time_series = np.arange(0, t_max, dt)
num_steps = time_series.shape[0]
th1 = np.zeros(num_steps)
th2 = np.zeros(num_steps)
vel1 = np.zeros(num_steps)
vel2 = np.zeros(num_steps)
acc1 = np.zeros(num_steps)
acc2 = np.zeros(num_steps)

# Initial conditions
th1[0] = th2[0] = np.pi/1.5

## Actual simulation

### Calculating $\ddot{\theta}_{1}$ and $\ddot{\theta}_{2}$

In [40]:
def get_acc(i):
    dth = th1[i - 1] - th2[i - 1]
    N = 1 + A * sin(dth) ** 2
    
    acc1 = (
        -(
            (1 + A) * C * sin(th1[i - 1])
            + A * B * vel1[i - 1] ** 2 * sin(dth)
            + A
            * cos(dth)
            * (vel1[i - 1] ** 2 * sin(dth) - C * sin(th2[i - 1]))
        )
        / N
    )
    
    acc2 = (
        (1 + A) * (vel1[i - 1] ** 2 * sin(dth) - C * sin(th2[i - 1]))
        + cos(dth)
        * ((1 + A) * C * sin(th1[i - 1]) + A * B * vel2[i - 1] ** 2 * sin(dth))
    ) / (B * N)
    
    return acc1, acc2

In [41]:
for i, t in enumerate(time_series[1:-1], start=1):
    acc1[i], acc2[i] = get_acc(i)
    vel1[i] = vel1[i - 1] + acc1[i] * dt
    vel2[i] = vel2[i - 1] + acc2[i] * dt
    th1[i] = th1[i - 1] + vel1[i] * dt
    th2[i] = th2[i - 1] + vel2[i] * dt

In [42]:
# Convenient vars
bob1x = L1 * np.sin(th1)
bob1y = -L1 * np.cos(th1)
bob2x = bob1x + L2 * np.sin(th2)
bob2y = bob1y - L2 * np.cos(th2)

In [43]:
# Save data
bob1_pos = np.stack((bob1x, bob1y))
bob2_pos = np.stack((bob2x, bob2y))
Ls = np.array([L1, L2])
Ms = np.array([m1, m2])
np.savez("data_temp", time_series=time_series, th1=th1, th2=th2, bob1_pos=bob1_pos, bob2_pos=bob2_pos, Ls=Ls, Ms=Ms)

## Graphics

In [44]:
# Don't show plot now
plt.ioff()

# Some colors
xred = "#bd4242"
xblue = "#4268bd"
xgreen = "#52B256"
xpurple = "#7F52b2"
xorange = "#fd9337"
xgrey = "550000"
xred_soft = "#bd7171"
xblue_soft = "#798ebd"

# General
plt.rcParams.update({"text.usetex": True, "font.family": "Helvetica"})
fig = plt.figure(figsize=(10, 9), layout="constrained")
gs = GridSpec(2, 2, figure=fig)
ax_vis = fig.add_subplot(gs[0, 0])
ax_th1_vs_th2 = fig.add_subplot(gs[0, 1])
ax_time = fig.add_subplot(gs[1, :])
fig.suptitle("Double pendulum", fontsize=25)

# Visual
ax_vis.set_title("Visual view", fontsize=20)
ax_vis.get_xaxis().set_ticks([])
ax_vis.get_yaxis().set_ticks([])
ax_vis.set_xlim(-1.25 * (L1+L2), 1.25 * (L1+L2))
ax_vis.set_ylim(-1.25 * (L1+L2), 1.25 * (L1+L2))

# th1 vs th2
ax_th1_vs_th2.set_title(r"$\theta_{1}$ vs. $\theta_{2}$", fontsize=20)
ax_th1_vs_th2.set_xlabel(r"$\theta_{1}$\ [rad]", fontsize=15)
ax_th1_vs_th2.set_ylabel(r"$\theta_{2}$\ [rad]", fontsize=15)
ax_th1_vs_th2.set_xlim(np.min(th1)-0.5, np.max(th1)+0.5)
ax_th1_vs_th2.set_ylim(np.min(th2)-0.5, np.max(th2)+0.5)

# Time plot
ax_time.set_title("Time plot", fontsize=20)
ax_time.set_xlabel(r"$t$\ [s]", fontsize=15)
ax_time.set_ylabel(r"$\theta_{1,2}$", fontsize=15)
ax_time.set_xlim(0, t_max)
ax_time.set_ylim(np.min(th1+th2)-0.5, np.max(th1+th2)+0.5)

(-12.24785829179098, 15.095429466727417)

In [45]:
#imgs = []
def animate(frame):
    for artist in ax_vis.lines + ax_vis.collections:
        artist.remove()
    
    trace1, = ax_vis.plot(
        bob1x[0:frame],
        bob1y[0:frame],
        "-o",
        color=xred_soft,
    )
    trace2, = ax_vis.plot(
        bob2x[0:frame],
        bob2y[0:frame],
        "-o",
        color=xblue_soft,
    )
    ln1, = ax_vis.plot(
        [0, bob1x[frame]],
        [0, bob1y[frame]],
        color="black",
        lw=3,
    )
    ln2, = ax_vis.plot(
        [bob1x[frame], bob2x[frame]],
        [bob1y[frame], bob2y[frame]],
        color="black",
        lw=3,
    )
    bob1, = ax_vis.plot(
        bob1x[frame],
        bob1y[frame],
        "o",
        markersize=22,
        color=xred,
        zorder=100,
    )
    bob2, = ax_vis.plot(
        bob2x[frame],
        bob2y[frame],
        "o",
        markersize=22,
        color=xblue,
        zorder=100,
    )
    (th1_vs_th2,) = ax_th1_vs_th2.plot(th1[:frame], th2[:frame], xpurple)
    (time_series1,) = ax_time.plot(time_series[:frame], th1[:frame], xred)
    (time_series2,) = ax_time.plot(time_series[:frame], th2[:frame], xblue)

    #imgs.append([ln1, ln2, bob1, bob2, th1_vs_th2, time_series1, time_series2])

In [46]:
anim = FuncAnimation(fig, animate, frames=num_steps, interval=10, blit=True)
HTML(anim.to_html5_video())